In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import sys
sys.path.append("/data2/saikiran.tedla/hdrvideo/diff")


from PIL import Image
from diffsynth import save_video, VideoData, load_state_dict
from diffsynth.pipelines.wan_video_new import WanVideoPipeline, ModelConfig
from modelscope import dataset_snapshot_download

from diffsynth.trainers.stuttgart_dataset import StuttgartDataset





dataset = StuttgartDataset(
    base_path="/data2/saikiran.tedla/hdrvideo/diff/data/stuttgart/carousel_fireworks_02",
    repeat=1,
    main_data_operator=StuttgartDataset.default_video_operator(
        base_path="/data2/saikiran.tedla/hdrvideo/diff/data/stuttgart/carousel_fireworks_02",
        max_pixels=1280*720,
        height=480,
        width=832,
        height_division_factor=16,
        width_division_factor=16,
        num_frames=13,
        time_division_factor=4,
        time_division_remainder=1,
    ),
)

# Get a sample from the dataset
data = dataset[0]
condition_video = data["video"]
save_video(condition_video, "stuttgart_input.mp4", fps=15)

pipe = WanVideoPipeline.from_pretrained(
    torch_dtype=torch.bfloat16,
    device="cuda",
    model_configs=[
        ModelConfig(model_id="Wan-AI/Wan2.2-TI2V-5B", origin_file_pattern="models_t5_umt5-xxl-enc-bf16.pth", offload_device="cpu", skip_download=True),
        ModelConfig(model_id="Wan-AI/Wan2.2-TI2V-5B", origin_file_pattern="diffusion_pytorch_model*.safetensors", offload_device="cpu", skip_download=True),
        ModelConfig(model_id="Wan-AI/Wan2.2-TI2V-5B", origin_file_pattern="Wan2.2_VAE.pth", offload_device="cpu", skip_download=True),
    ],
)
state_dict = load_state_dict("/data2/saikiran.tedla/hdrvideo/diff/models/train/firstweek1e4/checkpoints/epoch-25.safetensors")
pipe.dit.load_state_dict(state_dict)
pipe.enable_vram_management()

video = pipe(
    prompt="",
    #negative_prompt="色调艳丽，过曝，静态，细节模糊不清，字幕，风格，作品，画作，画面，静止，整体发灰，最差质量，低质量，JPEG压缩残留，丑陋的，残缺的，多余的手指，画得不好的手部，画得不好的脸部，畸形的，毁容的，形态畸形的肢体，手指融合，静止不动的画面，杂乱的背景，三条腿，背景人很多，倒着走",
    condition_video=condition_video[0:5],  # Use the first 5 frames as condition
    num_frames=13,
    seed=1, tiled=False,
    cfg_scale=1.0
)

RouteByType operator_map: [(<class 'str'>, <diffsynth.trainers.stuttgart_dataset.DataProcessingPipeline object at 0x14eea43e2a40>)]
MAX: 604.0 min: -0.09844971
Image.shape (1080, 1920, 3) max value: 16.0
tensor shape: torch.Size([3, 1080, 1920]) max value: tensor(16.)
MAX: 581.0 min: -0.10961914
Image.shape (1080, 1920, 3) max value: 16.0
tensor shape: torch.Size([3, 1080, 1920]) max value: tensor(16.)
MAX: 630.0 min: -0.11773682
Image.shape (1080, 1920, 3) max value: 16.0
tensor shape: torch.Size([3, 1080, 1920]) max value: tensor(16.)
MAX: 630.0 min: -0.070495605
Image.shape (1080, 1920, 3) max value: 16.0
tensor shape: torch.Size([3, 1080, 1920]) max value: tensor(16.)


Saving video:   8%|▊         | 1/13 [00:00<00:02,  4.41it/s]Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Conve

Loading models from: ./models/Wan-AI/Wan2.2-TI2V-5B/models_t5_umt5-xxl-enc-bf16.pth
    model_name: wan_video_text_encoder model_class: WanTextEncoder
    The following models are loaded: ['wan_video_text_encoder'].
Loading models from: ['./models/Wan-AI/Wan2.2-TI2V-5B/diffusion_pytorch_model-00003-of-00003-bf16.safetensors', './models/Wan-AI/Wan2.2-TI2V-5B/diffusion_pytorch_model-00002-of-00003-bf16.safetensors', './models/Wan-AI/Wan2.2-TI2V-5B/diffusion_pytorch_model-00001-of-00003-bf16.safetensors']
    model_name: wan_video_dit model_class: WanModel
        This model is initialized with extra kwargs: {'has_image_input': False, 'patch_size': [1, 2, 2], 'in_dim': 48, 'dim': 3072, 'ffn_dim': 14336, 'freq_dim': 256, 'text_dim': 4096, 'out_dim': 48, 'num_heads': 24, 'num_layers': 30, 'eps': 1e-06, 'seperated_timestep': True, 'require_clip_embedding': False, 'require_vae_embedding': False, 'fuse_vae_embedding_in_latents': True}
    The following models are loaded: ['wan_video_dit'].
Loa

100%|██████████| 50/50 [00:42<00:00,  1.19it/s]


In [4]:
pil_video = pipe.vae_output_to_video(video)
save_video(pil_video, "stuttgart_output.mp4", fps=15, quality=5)
# #write out video as npy array
import numpy as np
video_cpu = video.cpu().to(torch.float32).numpy()[0]

np.save("stuttgart_output.npy", video_cpu)

Saving video:   0%|          | 0/13 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Saving video: 100%|██████████| 13/13 [00:00<00:00, 22.77it/s]
